In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import gzip

# data path
path_csi =  'J:\\Data\\Wi-Fi_processed\\'

# data info
# data path
path_csi =  'J:\\Data\\Wi-Fi_processed\\'
# data info
df_info = pd.read_csv('data_subc_sig.csv')

# make data generator
def gen_csi(df_info,id_num,len_num):

    for file in set(df_info.id.values):
        # read sample data
        # load and uncompress.
        with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
            data1 = pickle.load(f)
        data1_abs = np.abs(data1)
        data1_ph = np.angle(data1)
        data1_abs.shape

        # normalize through subc axis
        abs_sub = np.mean(np.abs(data1),axis=(0,2,3))
        data1_norm = data1/abs_sub[np.newaxis,:,np.newaxis,np.newaxis]

        # differentiation
        data1_grad = np.gradient(data1_norm,axis=0)
        # zero pad
        pad_len = len_num - data1_grad.shape[0]
        data1_pad = np.pad(data1_grad,((0,pad_len),(0,0),(0,0),(0,0)),'constant',constant_values=0)
        # reshape
        data1_resh = data1_pad.reshape(max_len,30,6)
        # Label
        data1_lab = df_info[df_info.id==file]['id_person'].values[0]
        # One hot
        data1_one = np.eye(id_num+1)[data1_lab]
        
        yield(data1_resh,data1_one)
# parameters
max_id = np.max(df_info['id_person'])
max_len = int(np.max(df_info['len']))
learning_rate = 0.1
training_epochs = 100
batch_size = 5
# Load Dataset
from sklearn.model_selection import train_test_split
tr_idx,te_idx = train_test_split(df_info.index,test_size=0.2,random_state=10)
#id_num = len(np.unique(df_info.id_person))
df_train = df_info.loc[tr_idx]
df_test = df_info.loc[te_idx]

C:\Users\herok\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
gen = lambda: (r for r in gen_csi(df_info,max_id,max_len))
train_dataset = tf.data.Dataset().from_generator(gen, (tf.float32,tf.int32)).batch(batch_size)

In [ ]:
# create general iterator
iterator = tf.data.Iterator.from_structure(train_dataset.output_types,train_dataset.output_shapes)
next_element = iterator.get_next()

# make datasets that we can initialize separately, but using the same structure via the common iterator
training_init_op = iterator.make_initializer(train_dataset)

In [ ]:
def CNNmodel(in_data):
    input2d = tf.reshape(in_data, [-1,max_len,30,6])
    conv1 = tf.layers.conv2d(
        inputs=input2d, 
        filters=20, 
        kernel_size=[3,3], 
        strides=(1,1),
        #padding="same", 
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(
        inputs=conv1, 
        pool_size=[100,2], 
        strides=(10,1))
        #padding="same")
    conv2 = tf.layers.conv2d(
        inputs=pool1, 
        filters=50, 
        kernel_size=[3,3], 
        strides=(1,1),
        #padding="same", 
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(
        inputs=conv2, 
        pool_size=[100,2], 
        strides=(10,1))
        #padding="same")
    pool_flat = tf.layers.flatten(pool2)
    fc = tf.layers.dense(
        inputs= pool_flat, units=500, activation=tf.nn.relu)
    output = tf.layers.dense(inputs=fc, units=max_id+1)    
    return output

In [ ]:
# define cost/loss & optimizer
logits = CNNmodel(next_element[0])
labels = next_element[1]
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=labels))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# accuracy
pred = tf.argmax(logits,1)
equal = tf.equal(pred,tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(equal,tf.float32))

#init
init_op = tf.global_variables_initializer()
training_init_op = iterator.make_initializer(train_dataset)

In [ ]:
# run the training
epochs = training_epochs
results = pd.DataFrame(columns=['loss','accuracy'])
with tf.Session() as sess:
    sess.run(init_op)
    sess.run(training_init_op)
    for i in range(epochs):
        l, _, acc = sess.run([loss, optimizer, accuracy])
        print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i+1, l, acc * 100))
        '''
        # now setup the validation run
        test_iters = 1
        # re-initialize the iterator, but this time with validation data
        sess.run(test_init_op)
        test_l, test_acc = sess.run([loss, accuracy])
        print("Epoch: {}, test loss: {:.3f}, test accuracy: {:.2f}%".format(i+1, test_l, test_acc * 100))
        '''
        results.loc[i] = [l,acc]

In [ ]:
max_len